<a href="https://colab.research.google.com/github/camilagonzalezquevedo/Tarea_2_Mec_Estadist/blob/main/P2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install matplotlib

In [6]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Algorítmo Gray-Flip

Aplicación del algoritmo *gray-flip* para N spins, contiene todas las configuraciones posibles para un número N pequeño.

In [2]:
def gray_flip(tau, N):

    k = tau[0]  #valor inicial de k

    if k > N:
        return None, None

    tau[k-1] = tau[k]
    tau[k] = k + 1

    if k != 1:
        tau[0] = 1

    return tau, k  #devuelve el indice que se volteo

L = 2
N = L * L
tau_0 = list(range(1, N + 2))

spin = [-1]*N
print(spin)

for i in range(2 ** N - 1):
    tau_0, k = gray_flip(tau_0, N)
    spin[k - 1] = -spin[k - 1]
    print(spin)


[-1, -1, -1, -1]
[1, -1, -1, -1]
[1, 1, -1, -1]
[-1, 1, -1, -1]
[-1, 1, 1, -1]
[1, 1, 1, -1]
[1, -1, 1, -1]
[-1, -1, 1, -1]
[-1, -1, 1, 1]
[1, -1, 1, 1]
[1, 1, 1, 1]
[-1, 1, 1, 1]
[-1, 1, -1, 1]
[1, 1, -1, 1]
[1, -1, -1, 1]
[-1, -1, -1, 1]


# **Algorítmo Enumerate-Ising y Gray-Flip**

Ahora combinaremos ambos algoritmos para generar la densidad de estados de las redes cuadradas de dimensiones 2x2, 4x4 y 6x6 con y sin condiciones de borde periódica.

# **Para redes de 2x2 y 4x4**
*Con* condiciones periódicas

In [8]:
def gray_flip(tau, N):
    k = tau[0]
    if k > N:
        return None, None
    tau[k-1] = tau[k]
    tau[k] = k + 1
    if k != 1:
        tau[0] = 1
    return tau, k

#energia del modelo de Ising
def calculate_energy(spins, L, periodic=True):
    N = L * L
    spins = spins.reshape(L, L)
    energy = 0
    for i in range(L):
        for j in range(L):
            S = spins[i, j]
            neighbors = [spins[i, (j + 1) % L],  #derecha
                         spins[(i + 1) % L, j],  #arriba
                         spins[i, (j - 1) % L],  #izquierda
                         spins[(i - 1) % L, j]]  #abajo
            energy -= S * sum(neighbors)

    return energy / 2

#configuraciones y contar N(E)
def enumerate_ising(L, periodic=True):
    N = L * L
    spins = np.full(N, -1)
    tau = list(range(1, N + 2))
    energy_histogram = {}

    for _ in range(2 ** N):
        E = calculate_energy(np.array(spins), L, periodic)

        if E not in energy_histogram:
            energy_histogram[E] = 1
        else:
            energy_histogram[E] += 1

        tau, k = gray_flip(tau, N)
        if k is None:
            break

        spins[k-1] = -spins[k-1]

    return energy_histogram

#densidad de estados
def print_density_of_states(L, periodic=True):
    energy_histogram = enumerate_ising(L, periodic=True)
    sorted_energies = sorted(energy_histogram.keys())
    for energy in sorted_energies:
       if energy >= 0:
        print(f"E = {energy},  N(E) = {energy_histogram[energy]:,}")

#redes 2x2 y 4x4
for L in [2, 4]:
    print(f"\nDensidad de estados para {L}x{L}:")
    print_density_of_states(L)


Densidad de estados para 2x2:
E = 0.0,  N(E) = 12
E = 8.0,  N(E) = 2

Densidad de estados para 4x4:
E = 0.0,  N(E) = 20,524
E = 4.0,  N(E) = 13,568
E = 8.0,  N(E) = 6,688
E = 12.0,  N(E) = 1,728
E = 16.0,  N(E) = 424
E = 20.0,  N(E) = 64
E = 24.0,  N(E) = 32
E = 32.0,  N(E) = 2


Ahora para las redes de 2x2 y 4x4 *sin* condiciones periódicas:

In [9]:
def gray_flip(tau, N):
    k = tau[0]
    if k > N:
        return None, None
    tau[k-1] = tau[k]
    tau[k] = k + 1
    if k != 1:
        tau[0] = 1
    return tau, k

#energia del modelo de Ising
def calculate_energy_non_periodic(spins, L):
    N = L * L
    spins = spins.reshape(L, L)
    energy = 0
    for i in range(L):
        for j in range(L):
            S = spins[i, j]
            #vecinos solo si estan dentro de los limites
            if j + 1 < L:  #derecha
                energy -= S * spins[i, j + 1]
            if i + 1 < L:  #abajo
                energy -= S * spins[i + 1, j]
            if j - 1 >= 0:  #izquierda
                energy -= S * spins[i, j - 1]
            if i - 1 >= 0:  #arriba
                energy -= S * spins[i - 1, j]

    return energy / 2

#configuraciones y contar N(E)
def enumerate_ising_non_periodic(L):
    N = L * L
    spins = np.full(N, -1)
    tau = list(range(1, N + 2))
    energy_histogram = {}

    for _ in range(2 ** N):
        E = calculate_energy_non_periodic(np.array(spins), L)

        if E not in energy_histogram:
            energy_histogram[E] = 1
        else:
            energy_histogram[E] += 1

        tau, k = gray_flip(tau, N)
        if k is None:
            break

        spins[k-1] = -spins[k-1]

    return energy_histogram

#densidad de estados
def print_density_of_states_non_periodic(L):
    energy_histogram = enumerate_ising_non_periodic(L)
    sorted_energies = sorted(energy_histogram.keys())
    for energy in sorted_energies:
      if energy >= 0:
        print(f"E = {energy},  N(E) = {energy_histogram[energy]:,}")

#redes 2x2 y 4x4
for L in [2, 4]:
    print(f"\nDensidad de estados para {L}x{L}:")
    print_density_of_states_non_periodic(L)


Densidad de estados para 2x2:
E = 0.0,  N(E) = 12
E = 4.0,  N(E) = 2

Densidad de estados para 4x4:
E = 0.0,  N(E) = 11,472
E = 2.0,  N(E) = 9,984
E = 4.0,  N(E) = 7,344
E = 6.0,  N(E) = 4,928
E = 8.0,  N(E) = 2,638
E = 10.0,  N(E) = 1,216
E = 12.0,  N(E) = 584
E = 14.0,  N(E) = 224
E = 16.0,  N(E) = 72
E = 18.0,  N(E) = 32
E = 20.0,  N(E) = 8
E = 24.0,  N(E) = 2


# **Para redes de 6x6**

Con condiciones de borde periódicos:

In [5]:
def gray_flip(tau, N):
    k = tau[0]
    if k > N:
        return None, None
    tau[k-1] = tau[k]
    tau[k] = k + 1
    if k != 1:
        tau[0] = 1
    return tau, k

#energia del modelo de Ising
def calculate_energy(spins, L, periodic=True):
    N = L * L
    spins = spins.reshape(L, L)
    energy = 0
    for i in range(L):
        for j in range(L):
            S = spins[i, j]
            neighbors = [spins[i, (j + 1) % L],  #derecha
                         spins[(i + 1) % L, j],  #arriba
                         spins[i, (j - 1) % L],  #izquierda
                         spins[(i - 1) % L, j]]  #abajo
            energy -= S * sum(neighbors)

    return energy / 2

#configuraciones y contar N(E)
def enumerate_ising(L, periodic=True):
    N = L * L
    spins = np.full(N, -1)
    tau = list(range(1, N + 2))
    energy_histogram = {}

    for _ in range(2 ** N):
        E = calculate_energy(np.array(spins), L, periodic)

        if E not in energy_histogram:
            energy_histogram[E] = 1
        else:
            energy_histogram[E] += 1

        tau, k = gray_flip(tau, N)
        if k is None:
            break

        spins[k-1] = -spins[k-1]

    return energy_histogram

#densidad de estados
def print_density_of_states(L, periodic=True):
    energy_histogram = enumerate_ising(L, periodic=True)
    sorted_energies = sorted(energy_histogram.keys())
    for energy in sorted_energies:
       if energy >= 0:
        print(f"E = {energy},  N(E) = {energy_histogram[energy]:,}")


for L in [5]:
    print(f"\nDensidad de estados para {L}x{L}:")
    print_density_of_states(L)


Densidad de estados para 5x5:
E = 2.0,  N(E) = 7,413,900
E = 6.0,  N(E) = 5,276,500
E = 10.0,  N(E) = 2,645,740
E = 14.0,  N(E) = 1,024,150
E = 18.0,  N(E) = 314,300
E = 22.0,  N(E) = 82,750
E = 26.0,  N(E) = 14,800
E = 30.0,  N(E) = 2,470


Sin condiciones de borde periódicos:

In [7]:
def gray_flip(tau, N):
    k = tau[0]
    if k > N:
        return None, None
    tau[k-1] = tau[k]
    tau[k] = k + 1
    if k != 1:
        tau[0] = 1
    return tau, k

#energia del modelo de Ising
def calculate_energy_non_periodic(spins, L):
    N = L * L
    spins = spins.reshape(L, L)
    energy = 0
    for i in range(L):
        for j in range(L):
            S = spins[i, j]
            #vecinos solo si estan dentro de los limites
            if j + 1 < L:  #derecha
                energy -= S * spins[i, j + 1]
            if i + 1 < L:  #abajo
                energy -= S * spins[i + 1, j]
            if j - 1 >= 0:  #izquierda
                energy -= S * spins[i, j - 1]
            if i - 1 >= 0:  #arriba
                energy -= S * spins[i - 1, j]

    return energy / 2

#configuraciones y contar N(E)
def enumerate_ising_non_periodic(L):
    N = L * L
    spins = np.full(N, -1)
    tau = list(range(1, N + 2))
    energy_histogram = {}

    for _ in range(2 ** N):
        E = calculate_energy_non_periodic(np.array(spins), L)

        if E not in energy_histogram:
            energy_histogram[E] = 1
        else:
            energy_histogram[E] += 1

        tau, k = gray_flip(tau, N)
        if k is None:
            break

        spins[k-1] = -spins[k-1]

    return energy_histogram

#densidad de estados
def print_density_of_states_non_periodic(L):
    energy_histogram = enumerate_ising_non_periodic(L)
    sorted_energies = sorted(energy_histogram.keys())
    for energy in sorted_energies:
      if energy >= 0:
        print(f"E = {energy},  N(E) = {energy_histogram[energy]:,}")


for L in [5]:
    print(f"\nDensidad de estados para {L}x{L}:")
    print_density_of_states_non_periodic(L)


Densidad de estados para 5x5:
E = 0.0,  N(E) = 4,355,924
E = 2.0,  N(E) = 4,108,408
E = 4.0,  N(E) = 3,474,900
E = 6.0,  N(E) = 2,658,328
E = 8.0,  N(E) = 1,837,876
E = 10.0,  N(E) = 1,157,160
E = 12.0,  N(E) = 672,908
E = 14.0,  N(E) = 362,136
E = 16.0,  N(E) = 180,616
E = 18.0,  N(E) = 84,424
E = 20.0,  N(E) = 37,412
E = 22.0,  N(E) = 15,544
E = 24.0,  N(E) = 5,962
E = 26.0,  N(E) = 2,312
E = 28.0,  N(E) = 884
E = 30.0,  N(E) = 256
E = 32.0,  N(E) = 78
E = 34.0,  N(E) = 40
E = 36.0,  N(E) = 8
E = 40.0,  N(E) = 2
